In [110]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
#nltk.download()
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
import pickle

In [111]:
test_data = pd.read_csv("E:/NLP/test.csv")
test_data

id keyword location  \
0         0     NaN      NaN   
1         2     NaN      NaN   
2         3     NaN      NaN   
3         9     NaN      NaN   
4        11     NaN      NaN   
...     ...     ...      ...   
3258  10861     NaN      NaN   
3259  10865     NaN      NaN   
3260  10868     NaN      NaN   
3261  10874     NaN      NaN   
3262  10875     NaN      NaN   

                                                   text  
0                    Just happened a terrible car crash  
1     Heard about #earthquake is different cities, s...  
2     there is a forest fire at spot pond, geese are...  
3              Apocalypse lighting. #Spokane #wildfires  
4         Typhoon Soudelor kills 28 in China and Taiwan  
...                                                 ...  
3258  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...  
3259  Storm in RI worse than last hurricane. My city...  
3260  Green Line derailment in Chicago http://t.co/U...  
3261  MEG issues Hazardous Weather Outlook (HWO) htt...  
3262  #CityofCalgary has activated its Municipal Eme...  

[3263 rows x 4 columns]

In [112]:
P = test_data["text"]
P

0                      Just happened a terrible car crash
1       Heard about #earthquake is different cities, s...
2       there is a forest fire at spot pond, geese are...
3                Apocalypse lighting. #Spokane #wildfires
4           Typhoon Soudelor kills 28 in China and Taiwan
                              ...                        
3258    EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259    Storm in RI worse than last hurricane. My city...
3260    Green Line derailment in Chicago http://t.co/U...
3261    MEG issues Hazardous Weather Outlook (HWO) htt...
3262    #CityofCalgary has activated its Municipal Eme...
Name: text, Length: 3263, dtype: object

In [113]:
P = P.str.lower()
P

0                      just happened a terrible car crash
1       heard about #earthquake is different cities, s...
2       there is a forest fire at spot pond, geese are...
3                apocalypse lighting. #spokane #wildfires
4           typhoon soudelor kills 28 in china and taiwan
                              ...                        
3258    earthquake safety los angeles ûò safety faste...
3259    storm in ri worse than last hurricane. my city...
3260    green line derailment in chicago http://t.co/u...
3261    meg issues hazardous weather outlook (hwo) htt...
3262    #cityofcalgary has activated its municipal eme...
Name: text, Length: 3263, dtype: object

In [114]:
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [115]:
STOPWORDS = set(stopwordlist)
def cleaning_stopwords(P):
    return " ".join([word for word in str(P).split() if word not in STOPWORDS])
P = P.apply(lambda x: cleaning_stopwords(x))
P

0                             happened terrible car crash
1       heard #earthquake different cities, stay safe ...
2       forest fire spot pond, geese fleeing across st...
3                apocalypse lighting. #spokane #wildfires
4                  typhoon soudelor kills 28 china taiwan
                              ...                        
3258    earthquake safety los angeles ûò safety faste...
3259    storm ri worse last hurricane. city&amp;3other...
3260    green line derailment chicago http://t.co/utbx...
3261    meg issues hazardous weather outlook (hwo) htt...
3262    #cityofcalgary activated municipal emergency p...
Name: text, Length: 3263, dtype: object

In [116]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(P):
    translator = str.maketrans('', '', punctuations_list)
    return P.translate(translator)
P = P.apply(lambda x: cleaning_punctuations(x))
P

0                             happened terrible car crash
1       heard earthquake different cities stay safe ev...
2       forest fire spot pond geese fleeing across str...
3                   apocalypse lighting spokane wildfires
4                  typhoon soudelor kills 28 china taiwan
                              ...                        
3258    earthquake safety los angeles ûò safety faste...
3259    storm ri worse last hurricane cityamp3others h...
3260      green line derailment chicago httptcoutbxlcbiuy
3261    meg issues hazardous weather outlook hwo httpt...
3262    cityofcalgary activated municipal emergency pl...
Name: text, Length: 3263, dtype: object

In [117]:
def cleaning_repeating_char(P):
    return re.sub(r'(.)\1+', r'\1', P)
P = P.apply(lambda x: cleaning_repeating_char(x))
P

0                               hapened terible car crash
1       heard earthquake diferent cities stay safe eve...
2       forest fire spot pond gese fleing acros stret ...
3                   apocalypse lighting spokane wildfires
4                    typhon soudelor kils 28 china taiwan
                              ...                        
3258    earthquake safety los angeles ûò safety faste...
3259    storm ri worse last huricane cityamp3others ha...
3260        gren line derailment chicago htptcoutbxlcbiuy
3261    meg isues hazardous weather outlok hwo htptco3...
3262    cityofcalgary activated municipal emergency pl...
Name: text, Length: 3263, dtype: object

In [118]:
def cleaning_URLs(P):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',P)
P = P.apply(lambda x: cleaning_URLs(x))
P

0                               hapened terible car crash
1       heard earthquake diferent cities stay safe eve...
2       forest fire spot pond gese fleing acros stret ...
3                   apocalypse lighting spokane wildfires
4                    typhon soudelor kils 28 china taiwan
                              ...                        
3258    earthquake safety los angeles ûò safety faste...
3259    storm ri worse last huricane cityamp3others ha...
3260        gren line derailment chicago htptcoutbxlcbiuy
3261    meg isues hazardous weather outlok hwo htptco3...
3262    cityofcalgary activated municipal emergency pl...
Name: text, Length: 3263, dtype: object

In [119]:
def cleaning_numbers(P):
    return re.sub('[0-9]+', '', P)
P = P.apply(lambda x: cleaning_numbers(x))
P

0                               hapened terible car crash
1       heard earthquake diferent cities stay safe eve...
2       forest fire spot pond gese fleing acros stret ...
3                   apocalypse lighting spokane wildfires
4                      typhon soudelor kils  china taiwan
                              ...                        
3258    earthquake safety los angeles ûò safety faste...
3259    storm ri worse last huricane cityampothers har...
3260        gren line derailment chicago htptcoutbxlcbiuy
3261    meg isues hazardous weather outlok hwo htptcox...
3262    cityofcalgary activated municipal emergency pl...
Name: text, Length: 3263, dtype: object

In [120]:
tokenizer = RegexpTokenizer('\s+', gaps=True)
P = P.apply(tokenizer.tokenize)
P

0                          [hapened, terible, car, crash]
1       [heard, earthquake, diferent, cities, stay, sa...
2       [forest, fire, spot, pond, gese, fleing, acros...
3              [apocalypse, lighting, spokane, wildfires]
4                 [typhon, soudelor, kils, china, taiwan]
                              ...                        
3258    [earthquake, safety, los, angeles, ûò, safety...
3259    [storm, ri, worse, last, huricane, cityampothe...
3260    [gren, line, derailment, chicago, htptcoutbxlc...
3261    [meg, isues, hazardous, weather, outlok, hwo, ...
3262    [cityofcalgary, activated, municipal, emergenc...
Name: text, Length: 3263, dtype: object

In [121]:
st = nltk.PorterStemmer()
def stemming_on_text(P):
    P = [st.stem(word) for word in P]
    return P
P = P.apply(lambda x: stemming_on_text(x))
P

0                             [hapen, teribl, car, crash]
1       [heard, earthquak, difer, citi, stay, safe, ev...
2       [forest, fire, spot, pond, gese, fle, acro, st...
3                     [apocalyps, light, spokan, wildfir]
4                  [typhon, soudelor, kil, china, taiwan]
                              ...                        
3258    [earthquak, safeti, lo, angel, ûò, safeti, fa...
3259    [storm, ri, wors, last, hurican, cityampoth, h...
3260      [gren, line, derail, chicago, htptcoutbxlcbiuy]
3261    [meg, isu, hazard, weather, outlok, hwo, htptc...
3262    [cityofcalgari, activ, municip, emerg, plan, y...
Name: text, Length: 3263, dtype: object

In [122]:
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(P):
    P = [lm.lemmatize(word) for word in P]
    return P
P = P.apply(lambda x: lemmatizer_on_text(x))
P

0                             [hapen, teribl, car, crash]
1       [heard, earthquak, difer, citi, stay, safe, ev...
2       [forest, fire, spot, pond, gese, fle, acro, st...
3                     [apocalyps, light, spokan, wildfir]
4                  [typhon, soudelor, kil, china, taiwan]
                              ...                        
3258    [earthquak, safeti, lo, angel, ûò, safeti, fa...
3259    [storm, ri, wors, last, hurican, cityampoth, h...
3260      [gren, line, derail, chicago, htptcoutbxlcbiuy]
3261    [meg, isu, hazard, weather, outlok, hwo, htptc...
3262    [cityofcalgari, activ, municip, emerg, plan, y...
Name: text, Length: 3263, dtype: object

In [123]:
def list_to_string(P):
    P = (' '.join(map(str, P)))
    return P
P = P.apply(lambda x: list_to_string(x))
P

0                                  hapen teribl car crash
1            heard earthquak difer citi stay safe everyon
2       forest fire spot pond gese fle acro stret cano...
3                          apocalyps light spokan wildfir
4                        typhon soudelor kil china taiwan
                              ...                        
3258     earthquak safeti lo angel ûò safeti fasten xrwn
3259    storm ri wors last hurican cityampoth hardest ...
3260            gren line derail chicago htptcoutbxlcbiuy
3261      meg isu hazard weather outlok hwo htptcoxrbqjhn
3262       cityofcalgari activ municip emerg plan ycstorm
Name: text, Length: 3263, dtype: object

In [124]:
vectorizer = pickle.load(open("vectorizer.pickle", "rb"))

In [125]:
P=vectorizer.transform(P)

In [126]:
P.shape

(3263, 15452)

In [127]:
pickled_model = pickle.load(open('model.pkl', 'rb'))
Q=pd.DataFrame(pickled_model.predict(P))
Q.insert(0,'id',test_data.id,True)
Q.columns=['id','target']
Q

id  target
0         0       0
1         2       0
2         3       1
3         9       1
4        11       1
...     ...     ...
3258  10861       1
3259  10865       1
3260  10868       1
3261  10874       1
3262  10875       1

[3263 rows x 2 columns]

In [128]:
Q.to_excel('E:/NLP/Output_file.xlsx',index= False)